In [13]:
!pip install selenium
!pip install chromedriver-py==94.0.4606.41
!pip install requests

# libraries
import os
import pandas as pd
import shutil
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup as bs
from chromedriver_py import binary_path
import requests
import warnings; warnings.filterwarnings("ignore")

# unhide all rows and columns
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [14]:
years = list(range(2013,2024))
url_start = "https://www.basketball-reference.com/leagues/NBA_{}.html"

In [16]:
dfs = []

for year in years:
    url = url_start.format(year)
    data = requests.get(url)
    
    with open("season/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [19]:
for year in years:
    with open("season/{}.html".format(year)) as f:
        page = f.read()
        
    soup = bs(page, "html.parser")
    soup.find("tr", class_ = "over_header").decompose()
    team_stats_table = soup.find_all(id = "per_game-team")
    team_stat = pd.read_html(str(team_stats_table))[0]
    team_stat["Year"] = year
    
    dfs.append(team_stat)
    team_stats = pd.concat(dfs)

In [24]:
team_stats.tail(2)

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
29,30.0,Miami Heat*,82,241.5,39.2,85.3,0.460,12.0,34.8,0.344,27.3,50.5,0.540,19.1,23.0,0.831,9.7,30.9,40.6,23.8,8.0,3.0,13.5,18.5,109.5,2023
30,NaN,League Average,82,241.8,42.0,88.3,0.475,12.3,34.2,0.361,29.6,54.1,0.548,18.4,23.5,0.782,10.4,33.0,43.4,25.3,7.3,4.7,14.1,20.0,114.7,2023


In [28]:
team_stats = team_stats[team_stats["Team"] != 'League Average']

In [29]:
team_stats

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1.0,Denver Nuggets*,82,242.7,40.7,85.2,0.478,6.4,18.5,0.343,34.4,66.6,0.516,18.4,26.2,0.701,13.3,31.7,45.0,24.4,9.3,6.5,15.3,20.5,106.1,2013
1,2.0,Houston Rockets*,82,241.2,38.1,82.7,0.461,10.6,28.9,0.366,27.5,53.8,0.511,19.2,25.5,0.754,11.1,32.3,43.4,23.2,8.3,4.4,16.4,20.3,106.0,2013
2,3.0,Oklahoma City Thunder*,82,241.8,38.1,79.3,0.481,7.3,19.4,0.377,30.8,60.0,0.514,22.2,26.8,0.828,10.4,33.2,43.6,21.4,8.3,7.6,15.3,20.2,105.7,2013
3,4.0,San Antonio Spurs*,82,242.4,39.1,81.4,0.481,8.1,21.5,0.376,31.1,59.9,0.519,16.6,21.0,0.791,8.1,33.2,41.3,25.1,8.5,5.4,14.7,17.4,103.0,2013
4,5.0,Miami Heat*,82,242.4,38.4,77.4,0.496,8.7,22.1,0.396,29.6,55.4,0.536,17.4,23.0,0.754,8.2,30.4,38.6,23.0,8.7,5.4,13.9,18.7,102.9,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26.0,Orlando Magic,82,241.2,40.5,86.3,0.470,10.8,31.1,0.346,29.8,55.2,0.539,19.6,25.0,0.784,10.2,33.1,43.2,23.2,7.4,4.7,15.1,20.1,111.4,2023
26,27.0,Charlotte Hornets,82,241.8,41.3,90.4,0.457,10.7,32.5,0.330,30.5,57.9,0.528,17.6,23.6,0.749,11.0,33.5,44.5,25.1,7.7,5.2,14.2,20.3,111.0,2023
27,28.0,Houston Rockets,82,240.9,40.6,88.9,0.457,10.4,31.9,0.327,30.2,56.9,0.530,19.1,25.3,0.754,13.4,32.9,46.3,22.4,7.3,4.6,16.2,20.5,110.7,2023
28,29.0,Detroit Pistons,82,241.5,39.6,87.1,0.454,11.4,32.4,0.351,28.2,54.6,0.516,19.8,25.7,0.771,11.2,31.3,42.4,23.0,7.0,3.8,15.1,22.1,110.3,2023


In [30]:
team_stats['Team'] = team_stats['Team'].str.replace('*', '')

In [31]:
team_stats

,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1.0,Denver Nuggets,82,242.7,40.7,85.2,0.478,6.4,18.5,0.343,34.4,66.6,0.516,18.4,26.2,0.701,13.3,31.7,45.0,24.4,9.3,6.5,15.3,20.5,106.1,2013
1,2.0,Houston Rockets,82,241.2,38.1,82.7,0.461,10.6,28.9,0.366,27.5,53.8,0.511,19.2,25.5,0.754,11.1,32.3,43.4,23.2,8.3,4.4,16.4,20.3,106.0,2013
2,3.0,Oklahoma City Thunder,82,241.8,38.1,79.3,0.481,7.3,19.4,0.377,30.8,60.0,0.514,22.2,26.8,0.828,10.4,33.2,43.6,21.4,8.3,7.6,15.3,20.2,105.7,2013
3,4.0,San Antonio Spurs,82,242.4,39.1,81.4,0.481,8.1,21.5,0.376,31.1,59.9,0.519,16.6,21.0,0.791,8.1,33.2,41.3,25.1,8.5,5.4,14.7,17.4,103.0,2013
4,5.0,Miami Heat,82,242.4,38.4,77.4,0.496,8.7,22.1,0.396,29.6,55.4,0.536,17.4,23.0,0.754,8.2,30.4,38.6,23.0,8.7,5.4,13.9,18.7,102.9,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,26.0,Orlando Magic,82,241.2,40.5,86.3,0.470,10.8,31.1,0.346,29.8,55.2,0.539,19.6,25.0,0.784,10.2,33.1,43.2,23.2,7.4,4.7,15.1,20.1,111.4,2023
26,27.0,Charlotte Hornets,82,241.8,41.3,90.4,0.457,10.7,32.5,0.330,30.5,57.9,0.528,17.6,23.6,0.749,11.0,33.5,44.5,25.1,7.7,5.2,14.2,20.3,111.0,2023
27,28.0,Houston Rockets,82,240.9,40.6,88.9,0.457,10.4,31.9,0.327,30.2,56.9,0.530,19.1,25.3,0.754,13.4,32.9,46.3,22.4,7.3,4.6,16.2,20.5,110.7,2023
28,29.0,Detroit Pistons,82,241.5,39.6,87.1,0.454,11.4,32.4,0.351,28.2,54.6,0.516,19.8,25.7,0.771,11.2,31.3,42.4,23.0,7.0,3.8,15.1,22.1,110.3,2023


In [32]:
team_stats = team_stats.drop(team_stats.columns[0], axis=1)


In [33]:
team_stats

,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Denver Nuggets,82,242.7,40.7,85.2,0.478,6.4,18.5,0.343,34.4,66.6,0.516,18.4,26.2,0.701,13.3,31.7,45.0,24.4,9.3,6.5,15.3,20.5,106.1,2013
1,Houston Rockets,82,241.2,38.1,82.7,0.461,10.6,28.9,0.366,27.5,53.8,0.511,19.2,25.5,0.754,11.1,32.3,43.4,23.2,8.3,4.4,16.4,20.3,106.0,2013
2,Oklahoma City Thunder,82,241.8,38.1,79.3,0.481,7.3,19.4,0.377,30.8,60.0,0.514,22.2,26.8,0.828,10.4,33.2,43.6,21.4,8.3,7.6,15.3,20.2,105.7,2013
3,San Antonio Spurs,82,242.4,39.1,81.4,0.481,8.1,21.5,0.376,31.1,59.9,0.519,16.6,21.0,0.791,8.1,33.2,41.3,25.1,8.5,5.4,14.7,17.4,103.0,2013
4,Miami Heat,82,242.4,38.4,77.4,0.496,8.7,22.1,0.396,29.6,55.4,0.536,17.4,23.0,0.754,8.2,30.4,38.6,23.0,8.7,5.4,13.9,18.7,102.9,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Orlando Magic,82,241.2,40.5,86.3,0.470,10.8,31.1,0.346,29.8,55.2,0.539,19.6,25.0,0.784,10.2,33.1,43.2,23.2,7.4,4.7,15.1,20.1,111.4,2023
26,Charlotte Hornets,82,241.8,41.3,90.4,0.457,10.7,32.5,0.330,30.5,57.9,0.528,17.6,23.6,0.749,11.0,33.5,44.5,25.1,7.7,5.2,14.2,20.3,111.0,2023
27,Houston Rockets,82,240.9,40.6,88.9,0.457,10.4,31.9,0.327,30.2,56.9,0.530,19.1,25.3,0.754,13.4,32.9,46.3,22.4,7.3,4.6,16.2,20.5,110.7,2023
28,Detroit Pistons,82,241.5,39.6,87.1,0.454,11.4,32.4,0.351,28.2,54.6,0.516,19.8,25.7,0.771,11.2,31.3,42.4,23.0,7.0,3.8,15.1,22.1,110.3,2023


In [34]:
team_stats.to_cvs()

AttributeError: 'DataFrame' object has no attribute 'to_cvs'